# Capstone Project: The Battle of the Neighborhoods (Week 2)
### Applied Data Science Captstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#intro)
* [Data](#data)
* [Setting up](#setup)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name='introduction'></a>

It is well known that **pizza** is quite popular in **New York City**. With lots of pizzerias available throughout the city, stakeholders interested in opening a new parlor may encounter heavy competition. I will aim to use **data science** techniques to determine areas within the city with not too many options for pizza.

## Data <a name='data'></a>

The main data used will come from the **Foursquare Places API**. It allows developers to search explore venues based on a cetnered location. We can search for venues available within a radius for each neighborhood in NYC.

Additionally, I will use a **GeoJSON file** containing information about the neighborhoods in NYC to extract the borough, name, latitude and longitude values for each neighborhood. 

## Setting up <a name='setup'></a>

### Importing our Libraries

In [1]:
#pandas and numpy will be sued to read our .csv, json, and geojson files
import numpy as np 
import pandas as pd 
import json

#requests will be used to do our API reuests for the Foursquare API
import requests

#KMeans for neighborhood clustering
from sklearn.cluster import KMeans

#Mapping tool
import folium

#Set up pandas options to diaply the whole dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Collecting neighborhood Data

Now that we have all our libraries imported, we can collect our data. Let's load the data from **newyork_data.json** on a pandas dataframe.

In [2]:
#Load json
nyc_data = json.load(open('newyork_data.json'))

#Initialize dataframe with custom named columns
column_names = ['Borough','Name', 'Latitude', 'Longitude']
df_neighborhoods = pd.DataFrame(columns=column_names)

#Iterate through json features and append each neighborhood to the dataframe
for i, row in enumerate(nyc_data['features']):
    df_neighborhoods.loc[i,:] = ([row['properties']['borough'],
                                 row['properties']['name'],
                                 row['geometry']['coordinates'][1],
                                 row['geometry']['coordinates'][0]])
    
#Verify that we have 306 neighborhoods saved on the dataframe
print(df_neighborhoods.shape)

df_neighborhoods.head()

(306, 4)


,Borough,Name,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### API setup
We have our neighborhoods, now let's set up our API url. In order to use the Foursquare API, you need a **client ID, client secret and access_token** provided by <a href='https://foursquare.com/'>Foursquare</a> once you create a developer account. All my keys are saved on an external file called **keys.py** for security.

In [3]:
import keys

# Replace the values here for your own keys in order to call the API
CLIENT_ID = keys.CLIENT_ID
CLIENT_SECRET = keys.CLIENT_SECRET
ACCESS_TOKEN = keys.ACCESS_TOKEN
VERSION = '20180604'
LIMIT = 500
credentials = "client_id={}&client_secret={}&oauth_token={}".format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN)
parameters = "v={}&limit={}".format(VERSION, LIMIT)

search_url = "https://api.foursquare.com/v2/venues/explore?{}&{}".format(credentials, parameters)
print(search_url)

https://api.foursquare.com/v2/venues/explore?client_id=JHTTDJBJUFNGC3N1HBTYRWA5PBBIPAG04DOWOCMZACIGD2VS&client_secret=RRJE5IIXHSWYNP5ZFA2MXM40N4BNFU5AQQQQYAEZMGKEE1NM&oauth_token=31MZ5WDVPNAUOBFVXDBQPVARI2V31JM1VETVND4GAZOYILXX&v=20180604&limit=500


The only thing needed to get venue information is to add our query parameters. Let's create a method that, given a pandas dataframe with neighborhood data, makess an API call for each row's latitude and longitude values to get the venues within 500 meters within the center of those coordinates. 

In [4]:
#The categories field is composed of other children fields. We are only interested in the name
def get_cat_name(row):
    try:
        #Extract name of venue category
        return row['Category'][0]['name']
    except:
        #If there is no category available, return None
        return np.NaN


def get_nearby_venues(neighborhoods, url, radius): 
    
    #Initialize dataframe    
    df = pd.DataFrame()

    for b, n, lat, lon in zip(neighborhoods['Borough'],neighborhoods['Name'], neighborhoods['Latitude'], 
                              neighborhoods['Longitude']):

        search_query = url + "&ll={},{}&radius={}".format(lat, lon, radius)

        response = requests.get(search_query).json()["response"]['groups'][0]['items']  
        
        #Print neighborhood name and number of found venues
        print(n, len(response), 'venues')

        df = df.append([(b, n, v['venue']['name'],
                               v['venue']['categories'],
                               v['venue']['location']['lat'],
                               v['venue']['location']['lng']) for v in response])

    
    #Assign columns
    columns = ['Borough', 'Neighborhood', 'Name', 'Category', 'Latitude', 'Longitude']
    df.columns = columns
    
    #Extract category name from categories
    df['Category'] = df.apply(get_cat_name, axis=1)
    
    #Remove misleading venue categories
    df = df[~df['Category'].isin(['Food','Neighborhood'])]
    
    return df   

Our function it's ready. Let's make our call. It will take a couple of minutes to build the whole dataframe 

In [5]:
df_venues = get_nearby_venues(df_neighborhoods, search_url, 750)
df_venues.to_csv('venues.csv',index=False)

print(df_venues.shape)
df_venues.head()

Wakefield 65 venues
Co-op City 76 venues
Eastchester 73 venues
Fieldston 18 venues
Riverdale 75 venues
Kingsbridge 100 venues
Marble Hill 100 venues
Woodlawn 79 venues
Norwood 89 venues
Williamsbridge 63 venues
Baychester 82 venues
Pelham Parkway 65 venues
City Island 80 venues
Bedford Park 95 venues
University Heights 81 venues
Morris Heights 53 venues
Fordham 100 venues
East Tremont 90 venues
West Farms 78 venues
High  Bridge 58 venues
Melrose 97 venues
Mott Haven 56 venues
Port Morris 52 venues
Longwood 76 venues
Hunts Point 57 venues
Morrisania 51 venues
Soundview 68 venues
Clason Point 20 venues
Throgs Neck 45 venues
Country Club 59 venues
Parkchester 100 venues
Westchester Square 91 venues
Van Nest 60 venues
Morris Park 88 venues
Belmont 100 venues
Spuyten Duyvil 36 venues
North Riverdale 66 venues
Pelham Bay 85 venues
Schuylerville 72 venues
Edgewater Park 78 venues
Castle Hill 54 venues
Olinville 85 venues
Pelham Gardens 62 venues
Concourse 93 venues
Unionport 83 venues
Edenwal

KeyError: 'groups'

## Methodology <a name='methodology'></a>

We now have a dataframe with venue information in New York City. To find the best neighborhoods to open a pizza parlor, we will utilize the k-means algorith to group the neighborhoods in 5 clusters based on the list of most frequent venues by neighborhood.

In order to do so, we need to convert the neighborhood category data that is currently categorical to numerical values. For that we will apply **one-hot encoding** to our **Category** row. This will basically create a new column named after each different value for Category and the values will be 1 or 0.

In [ ]:
# Get the one-hot encodign dataframe
df_onehot = pd.get_dummies(df_venues['Category'], prefix='', prefix_sep='')

# Add Borough and Neighborhood to the dataframe. Since some neighborhoods in different boroughs are named the same. The borough will belp
# make that distinction
df_onehot['Borough'] = df_venues['Borough']
df_onehot['Neighborhood'] = df_venues['Neighborhood']

# Rearrange columns moving Borough and Neighborhood to the left
columns = list(df_onehot.columns[-2:]) + list(df_onehot.columns[:-2])
df_onehot = df_onehot[columns]

# Group by neighborhood
df_onehot = df_onehot.groupby(by=['Borough','Neighborhood']).mean()

df_onehot.reset_index(inplace=True)


print(df_onehot.shape)
df_onehot.head()

Now let's create a dataframe with the top 10 most frequent venues by neighborhood. 

In [ ]:
# Create our columns
indicators = ['st','nd','rd','th']
columns = ['Borough', 'Neighborhood'] + [str(i+1) + (indicators[i] if i < len(indicators)-1 else indicators[-1]) + ' Most Frequent Venue' for i in range(0,10)]

df_top10 = pd.DataFrame(columns=columns)

for b,n in zip(df_onehot['Borough'], df_onehot['Neighborhood']):
    
    # Get dataframe for specific neighborhood and transpose it
    df_n = df_onehot[(df_onehot['Neighborhood'] == n) & (df_onehot['Borough'] == b)]
    
    # Remove Neighborhood & Borough columns
    df_n = df_n.drop(['Borough','Neighborhood'], axis=1)
    
    # Transpose dataframe
    df_n = df_n.transpose().reset_index()
        
    # Rename columns
    df_n.columns = ['Venue', 'Frequency']
        
    # Get 10 most requent locations and keep only Venue
    df_n = df_n.sort_values(by='Frequency', ascending=False)[['Venue']].head(10) 
    
    # Transpose again, add neighborhood and rename columns
    df_n = df_n.transpose().reset_index(drop=True)
    
    # Add borough and neighborhood and move to the left
    df_n['Borough'] = b 
    df_n['Neighborhood'] = n    
    df_n = df_n[list(df_n.columns[-2:]) + list(df_n.columns[:-2])]
    
    df_n.columns = columns
    
    #Append to DataFrame
    df_top10 = df_top10.append(df_n)
    
    del(df_n)

print(df_top10.shape)
df_top10.head()

Now let's merge df_top10 and df_neighborhoods.

In [ ]:
# Our original nieghborhoods dataframe had Name instead of Neighborhood. Let's rename our columns to merge our datasets
df_neighborhoods.columns = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
df_cluster = pd.merge(df_neighborhoods, df_top10, on=['Borough','Neighborhood'])
df_cluster.head()

We are now ready to run kmeans clustering on our dataframe. For that, we will utilize the KMeans package from **sci-kit learn**.

In [ ]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_onehot.drop(['Borough','Neighborhood'], 1))

# Add cluster values to dataframe
df_cluster['Cluster'] = kmeans.labels_

## Analysis <a name='analysis'></a>

Our neighborhoods are now clustered. However, we haven't found our neighborhoods yet. Let's visualize our clusters on the map. For that we will create function that will create a Folium map object.

In [ ]:
# NYC Coordinates
latitude = 40.712
longitude = -74.005

import folium
from matplotlib import cm
import matplotlib.colors as colors
from shapely.geometry import Polygon

def map_clusters(df):

    cmap = list(cm.rainbow(np.linspace(0, 1, len(set(kmeans.labels_)))))
    cmap = [colors.rgb2hex(i) for i in cmap]
    map_nyc_clustered = folium.Map(location=[latitude, longitude], zoom_start=11.2, tiles='cartodbpositron')

    for neighborhood, cluster, lat, lon, in zip(df['Neighborhood'],
                                                df['Cluster'],
                                                df['Latitude'],
                                                df['Longitude']):

        label = folium.Popup('{}, {}'.format(neighborhood, cluster), parse_html = False)

        folium.CircleMarker(
                [lat, lon],
                radius=5,
                popup=label,
                color=None,
               fill=True,
                fill_color=cmap[cluster],
                fill_opacity=0.7,
                parse_html=False).add_to(map_nyc_clustered)
        
    return map_nyc_clustered

In [ ]:
map_clusters(df_cluster)

Let's understand a little better our clusters. We can group our dataframe by cluster count the appearances of Pizza Place as nth most frequent venue.

In [ ]:
new_columns = ['Pizza Place as ' + c for c in columns[2:]]
df_count = pd.DataFrame(columns=new_columns)

for c, nc in zip(columns[2:], new_columns):     
    values = df_cluster.groupby('Cluster')[c].value_counts().values.tolist()
    indices = [[c[0], values[i]] for i,c in enumerate(df_cluster.groupby('Cluster')[c].value_counts().index.tolist()) if 'Pizza Place' in c[1]]
    for i in indices:
        df_count.loc[i[0],nc] = i[1]

# Replace NaN values with 0
df_count = df_count.fillna(0)

# Sum all values into total
df_count['Total'] = df_count['Total'] = df_count[new_columns].sum(axis=1)

df_count.sort_values(by='Total', ascending=True, inplace=True)
df_count

## Results <a name='results'></a>
As we can see, the top clusters hae the least amount of pizzerias. The more rows we choose we can expand our options of neighborhoods. Let's stick with the top 3 clusters.

In [ ]:
map_clusters(df_cluster[df_cluster['Cluster'].isin(df_count.head(3).index.values.tolist())])

## Conclusion <a name='conclusion'></a>
KMeans clustering is a powerful algorithm that can provide us with good insights of how data is distributed. Of course, there are other factors that can improve our decision making such as demographics and psycographic data.